<h3 style='color:purple'>Exercise</h3>

1. Load heart disease dataset in pandas dataframe
1. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
1. Convert text columns to numbers using label encoding and one hot encoding
1. Apply scaling
1. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
1. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
1. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best. 

In [61]:
import pandas as pd

df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [62]:
z_scores = (df.Cholesterol - df.Cholesterol.mean()) / df.Cholesterol.std()
df_no_outliers = df[(z_scores > -3) & (z_scores < 3)]
df_no_outliers.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [63]:
df_new = pd.get_dummies(df_no_outliers, drop_first=True)
df_new.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,True,True,False,False,True,False,False,False,True
1,49,160,180,0,156,1.0,1,False,False,True,False,True,False,False,True,False
2,37,130,283,0,98,0.0,0,True,True,False,False,False,True,False,False,True
3,48,138,214,0,108,1.5,1,False,False,False,False,True,False,True,True,False
4,54,150,195,0,122,0.0,0,True,False,True,False,True,False,False,False,True


In [64]:
X = df_new.drop('HeartDisease', axis='columns')
y = df_new.HeartDisease

In [65]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
X_scaler

array([[-1.43736051,  0.40855699,  0.85062571, ..., -0.82395333,
        -0.9989077 ,  1.14736903],
       [-0.4804758 ,  1.4885213 , -0.16377238, ..., -0.82395333,
         1.00109349, -0.87155917],
       [-1.75632207, -0.13142516,  0.79478728, ..., -0.82395333,
        -0.9989077 ,  1.14736903],
       ...,
       [ 0.37008838, -0.13142516, -0.61978619, ...,  1.21366098,
         1.00109349, -0.87155917],
       [ 0.37008838, -0.13142516,  0.35738627, ..., -0.82395333,
         1.00109349, -0.87155917],
       [-1.65000155,  0.30056056, -0.2103044 , ..., -0.82395333,
        -0.9989077 ,  1.14736903]])

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=20)

In [67]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(), X, y, cv=5)
scores.mean()

np.float64(0.6841530054644809)

In [68]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(estimator=SVC(), n_estimators=100, max_samples=0.8, random_state=0)
scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()


np.float64(0.6852459016393443)

In [69]:
from sklearn.tree import DecisionTreeClassifier

dt_scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
dt_scores.mean()

np.float64(0.730054644808743)

In [70]:
dt_bag_model = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, max_samples=0.8, random_state=0)
scores = cross_val_score(dt_bag_model, X, y, cv=5)
scores.mean()

np.float64(0.8010928961748635)